In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import os, sys
from pymilvus import MilvusClient
sys.path.append('../pubmed_rag')
from utils import config_loader
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
# from model import (
#     mean_pooling
# )


Using this notebook to develop the prompt for Llama3.1. 

Requires that the DB already made.

In [1]:
vector_query = '''
what nodes and edges should be used for a knowledge graph about the gut microbiome?
'''
path_config = '/users/anglup/GitHub/pubmed-rag/config.yaml'
path_to_repo = '/users/anglup/github/pubmed-rag'

In [2]:
import os
path_run_search = os.path.join(path_to_repo, 'run_search.py')
! python "{path_run_search}" -c {path_config} -q "{vector_query}"

[2024-10-08 13:37:40,708] root: INFO - Path to log file: logs/2024108_133740_run_search.log
[2024-10-08 13:37:40,708] root: INFO - Path to config file: /users/anglup/GitHub/pubmed-rag/config.yaml
[2024-10-08 13:37:40,708] root: INFO - Loading config params ... 
[2024-10-08 13:37:40,709] root: INFO - Configuration: {'pmid file path': 'notebooks/data/test_pmid_list.csv', 'biocjson output path': 'biocjson', 'max_tokens': 300, 'transformer_model': 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', 'db name': '/users/anglup/github/pubmed-rag/demo.db', 'tsne': 'tsne.png', 'n_results': 10}
[2024-10-08 13:37:40,709] root: INFO - Accessing /users/anglup/github/pubmed-rag/demo.db
[2024-10-08 13:37:40,710] pymilvus.orm.connections: INFO - Pass in the local path /users/anglup/github/pubmed-rag/demo.db, and run it using milvus-lite
[2024-10-08 13:37:41,740] pymilvus.milvus_client.milvus_client: DEBUG - Created new connection using: 0cc013d73964497f9688b80ab826210a
[2024-10-08 13:37:41

In [3]:
# let's try this 
from run_search import main

In [6]:
results = main(
    path_config,
    vector_query,
)

Path to config file: /users/anglup/GitHub/pubmed-rag/config.yaml
Loading config params ... 
Configuration: {'pmid file path': 'notebooks/data/test_pmid_list.csv', 'biocjson output path': 'biocjson', 'max_tokens': 300, 'transformer_model': 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', 'db name': '/users/anglup/github/pubmed-rag/demo.db', 'tsne': 'tsne.png', 'n_results': 10}
Accessing /users/anglup/github/pubmed-rag/demo.db
Loading model microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext from HuggingFace


/Users/anglup/Library/Caches/pypoetry/virtualenvs/pubmed-rag-iB5Z1S4t-py3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embedding query: 
what nodes and edges should be used for a knowledge graph about the gut microbiome?

Searching against vector database


In [7]:
import json

In [8]:
for x in (results):
    print(json.dumps(x, indent=4))

{
    "id": 1011,
    "distance": 0.9812421202659607,
    "entity": {
        "section": "discuss",
        "pmid": 36258252,
        "text": "This pattern suggests a focus on label function construction and generative model training may be key steps to focus on in future work . Although we found that label functions cannot be re-used across all edge types with the standard task framing, strategies like multitask or transfer learning may make multi-label-function efforts more successful."
    }
}
{
    "id": 1010,
    "distance": 0.9812421202659607,
    "entity": {
        "section": "discuss",
        "pmid": 36258252,
        "text": "Discussion . Filling out knowledge bases via manual curation can be an arduous and erroneous task . Using manual curation alone becomes impractical as the rate of publications continuously increases . Data programming is a paradigm that uses label functions to speed up the annotation process and can be used to solve this problem . However, creating usef